In [ ]:
# Preamble
try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass


import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt

In [ ]:
# We read in files from 240219 to create references frames for the fake reference night 001122
night_reference    = '001122'
night_observations = '240219'

calibrations = dict()
calibrations['SimTh'] = dict()
calibrations['FibTh'] = dict()
calibrations['SimLC'] = dict()
calibrations['Flat'] = dict()
calibrations['Dark_1800'] = dict()
calibrations['Dark_900'] = dict()
calibrations['Dark_600'] = dict()

# SimTh for CCD1 with 180s
calibrations['SimTh']['ccd_1'] = ['0052','0053','0054','0055','0056','0170','0171','0172','0173','0174']
# SimTh for CCD2 with 60s
calibrations['SimTh']['ccd_2'] = ['0057','0058','0060','0061','0165','0166','0167','0168','0169'] # excluding bad 0059 because it had not ThXe in it.
# SimTh for CCD3 with 15s
calibrations['SimTh']['ccd_3'] = ['0062','0063','0064','0065','0066','0160','0161','0162','0163','0164']

# FibTh for CCD1
calibrations['FibTh']['ccd_1'] = ['0047','0048','0049','0050','0051','0217','0218','0219','0220','0221','0222','0223']
# FibTh for CCD2
calibrations['FibTh']['ccd_2'] = ['0042','0043','0044','0045','0046','0210','0211','0212','0213','0214','0215','0216']
# FibTh for CCD3
calibrations['FibTh']['ccd_3'] = ['0037','0038','0039','0040','0041','0086','0087','0122','0123','0203','0204','0205','0206','0207','0208','0209']

# SimLC for CCD2
calibrations['SimLC']['ccd_2'] = ['0002','0003','0004','0005','0006','0007','0008','0067','0068','0072','0073','0074','0075','0076','0077','0078','0155','0156','0157','0158','0159']
# SimLC for CCD3
calibrations['SimLC']['ccd_3'] = ['0002','0003','0004','0005','0006','0007','0008','0067','0068','0072','0073','0074','0075','0076','0077','0078','0155','0156','0157','0158','0159']

# Flat for CCD1
calibrations['Flat']['ccd_1'] = ['0030','0031','0032','0033','0034','0035','0036','0196','0197','0198','0199','0200','0201','0202']
# Flat for CCD2
calibrations['Flat']['ccd_2'] = ['0016','0017','0018','0019','0020','0021','0022','0182','0183','0184','0185','0186','0187','0188']
# Flat for CCD3
calibrations['Flat']['ccd_3'] = ['0009','0010','0011','0012','0013','0014','0015','0175','0176','0177','0178','0179','0180','0181']

# Dark 1800s for CCD1
calibrations['Dark_1800']['ccd_1'] = ['0224','0225','0226']
# Dark 1800s for CCD2
calibrations['Dark_1800']['ccd_2'] = ['0224','0225','0226']
# Dark 1800s for CCD3
calibrations['Dark_1800']['ccd_3'] = ['0224','0225','0226']

# Dark 900s for CCD1
calibrations['Dark_900']['ccd_1'] = ['0227','0228','0229']
# Dark 900s for CCD2
calibrations['Dark_900']['ccd_2'] = ['0227','0228','0229']
# Dark 900s for CCD3
calibrations['Dark_900']['ccd_3'] = ['0227','0228','0229']

# Dark 300s for CCD1
calibrations['Dark_600']['ccd_1'] = ['0230','0231','0232']
# Dark 300s for CCD2
calibrations['Dark_600']['ccd_2'] = ['0230','0231','0232']
# Dark 300s for CCD3
calibrations['Dark_600']['ccd_3'] = ['0230','0231','0232']

run_integer = 1

for calibration in calibrations.keys():

    for ccd in [1,2,3]:

        if f'ccd_{ccd}' in calibrations[calibration].keys():
        
            ref_run = f"{run_integer:04d}"
            runs = calibrations[calibration][f'ccd_{ccd}']

            print(calibration, ccd, ref_run, runs, len(runs))

            ref_fits = fits.open('/Users/buder/velocereduction/observations/' + night_observations + '/ccd_'+str(ccd)+'/19feb'+str(ccd)+runs[0]+'.fits')

            images = [ref_fits[0].data]
            header = ref_fits[0].header

            ref_fits.close()

            plt.imshow(images[0], vmin=np.percentile(images[0],5), vmax=np.percentile(images[0],99.5), cmap='gray')
            plt.colorbar()
            plt.title(calibration+' CCD'+str(ccd)+' night '+night_observations+' run '+runs[0])
            plt.show()
            plt.close()

            for run in runs[1:]:
                next_fits = fits.open('/Users/buder/velocereduction/observations/' + night_observations + '/ccd_'+str(ccd)+'/19feb'+str(ccd)+run+'.fits')
                images.append(next_fits[0].data)

                plt.imshow(next_fits[0].data, vmin=np.percentile(next_fits[0].data,5), vmax=np.percentile(next_fits[0].data,99.5), cmap='gray')
                plt.colorbar()
                plt.title(calibration+' CCD'+str(ccd)+' night '+night_observations+' run '+run)
                plt.savefig('/Users/buder/ccd_'+str(ccd)+'_'+calibration+'_night_'+night_observations+'_run_'+run+'.png')
                plt.show()
                plt.close()

                next_fits.close()

            median_image = np.array(np.nanmedian(np.array(images, dtype=np.uint16), axis=0), dtype=np.uint16)
            plt.imshow(median_image, vmin=np.percentile(median_image,5), vmax=np.percentile(median_image,99.5), cmap='gray')
            plt.colorbar()
            plt.title('Median '+calibration+' image CCD'+str(ccd)+' night '+night_observations)
            plt.savefig('/Users/buder/ccd_'+str(ccd)+'_'+calibration+'_night_'+night_observations+'_run_median.png')
            plt.show()

            fits.writeto('/Users/buder/velocereduction/observations/' + night_reference + '/ccd_'+str(ccd)+'/22nov'+str(ccd)+ref_run+'.fits', median_image, header, overwrite=True)

            run_integer += 1
